<h2>KNN - K Vizinhos Mais Próximos<h2>


Algoritmo de classificação de dados, o KNN (K-Nearest Neighbors) busca categorizar cada amostra em um conjunto de dados ao avaliar sua proximidade em relação aos K vizinhos mais próximos. Caso a maioria desses vizinhos pertença a uma classe específica, a amostra em questão será classificada nessa categoria.<br>

Vale ressaltar que o KNN é um algoritmo supervisionado, exigindo um conjunto de dados previamente rotulado para treinamento. É fundamental destacar que a escolha do valor de K é crucial, pois impacta diretamente na eficiência do algoritmo. Valores menores podem resultar em classificações instáveis, enquanto valores maiores podem levar a uma perda de detalhes importantes na discriminação entre classes. Ademais, o KNN é sensível a outliers, pois a presença de dados atípicos pode afetar significativamente as decisões de classificação. <br>

Além disso, o desempenho do KNN pode ser influenciado pela métrica de distância escolhida, sendo a distância euclidiana a mais comum. <br>

In [2]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

dados = load_breast_cancer()

x_can = pd.DataFrame(dados.data, columns= [dados.feature_names])
y_can = pd.Series(dados.target)

y_can.value_counts() # Dados bem balanceados

1    357
0    212
Name: count, dtype: int64

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Normalizando as váriaveis preditoras, pois estamos trabalhando com distâncias de váriaveis diferentes
normalizador = MinMaxScaler(feature_range=(0,1))
X_norm = normalizador.fit_transform(x_can)

#Treino e teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X_norm, y_can, test_size= 0.3, random_state=16)

#Criação do Modelo
modelo_knn_1 = KNeighborsClassifier(n_neighbors=5)
modelo_knn_1.fit(X_treino, Y_treino)

#Score
resultado = modelo_knn_1.score(X_teste,Y_teste)
print('Acurácia:', resultado)

Acurácia: 0.9649122807017544


<h2>Ajustando Parâmetros<h2>

No KNN podemos ajustar o parâmetro $K$ para encontrar o melhor número de vizinhos para o modelo, assim como podemos variar o calculo da distância a ser usada pelo modelo, podendo ser Euclidiana, Manhattan, Minkowski, etc.

In [11]:
import numpy as np  
from sklearn.model_selection import GridSearchCV

valores_K = np.array([3,5,7,9,11])
distancia = ['minkowski','chebyshev'] #https://scikit-learn.org/0.24/modules/generated/sklearn.neighbors.DistanceMetric.html
valores_p = np.array([1, 2,3, 4]) # p = 2 é equivalente a distancia euclidiana e p = 1 é equivalente a Manhattan, usando a formula da minkowski
valores_grid = {'n_neighbors': valores_K, 'metric': distancia, 'p':valores_p}

# Modelo
modelo_grid = KNeighborsClassifier()

#Criando Grids
GriddKNN = GridSearchCV(estimator=modelo_grid, param_grid= valores_grid, cv = 5 )
GriddKNN.fit(X_norm,y_can)

# Imprimindo melhores parâmetros
print('Melhor Acurácia:', GriddKNN.best_score_)
print('Melhor K:', GriddKNN.best_estimator_.n_neighbors)
print('Método Distância:', GriddKNN.best_estimator_.metric)
print('Melhor Valor de p:', GriddKNN.best_estimator_.p)

Melhor Acurácia: 0.9701133364384411
Melhor K: 3
Método Distância: minkowski
Melhor Valor de p: 1


<h2>Banco de Dados Iris<h2>

Iris é uma de flor onde podem ser de três tipos, a Versicolor, Setosa e Virginica. Com bases nas caracteristicas de cada um dos tipos de Iris será usado o algoritmo de classificação KNN para identificar qual grupo uma flor nova corresponde com base nas suas caracterisicas, sendo elas o comprimento e largura da sepala e da pétala em centimetros.

In [12]:
from sklearn.datasets import load_iris
iris = load_iris()
x_iris = pd.DataFrame(iris.data, columns= [iris.feature_names])
y_iris = pd.Series(iris.target)

x_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [13]:
# Normalizando as váriaveis preditoras, pois estamos trabalhando com distâncias de váriaveis diferentes
normalizador_iris = MinMaxScaler(feature_range=(0,1))
X_norm_iris = normalizador_iris.fit_transform(x_iris)

# Modelo
modelo_grid_iris = KNeighborsClassifier()

#Criando Grids
GriddKNN_iris = GridSearchCV(estimator=modelo_grid_iris, param_grid= valores_grid, cv = 5)
GriddKNN_iris.fit(X_norm_iris,y_iris)

# Imprimindo melhores parâmetros
print('Melhor Acurácia:', GriddKNN_iris.best_score_)
print('Melhor K:', GriddKNN_iris.best_estimator_.n_neighbors)
print('Método Distância:', GriddKNN_iris.best_estimator_.metric)
print('Melhor Valor de p:', GriddKNN_iris.best_estimator_.p)

Melhor Acurácia: 0.9733333333333334
Melhor K: 5
Método Distância: minkowski
Melhor Valor de p: 3


In [35]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species",
                 size='petal_length', hover_data=['petal_width'])
fig.show()

In [39]:
import plotly.express as px

#Treino e teste
X_treino_iris, X_teste_iris, Y_treino_iris, Y_teste_iris = train_test_split(X_norm_iris, y_iris, test_size= 0.3, random_state=10)

# Fit the model on training data, predict on test data
clf = KNeighborsClassifier(n_neighbors=5, p=3, metric= 'minkowski')
clf.fit(X_treino_iris, Y_treino_iris)
y_score = clf.predict_proba(X_teste_iris)[:, 1]

fig = px.scatter(
    X_teste_iris, x=0, y=1,
    color=y_score, color_continuous_scale='RdBu',
    symbol=Y_teste_iris, symbol_map={'0': 'star', '1': 'circle', '2': 'diamond'},
    labels={'symbol': 'label', 'color': 'score of <br>first class'}
)
fig.update_traces(marker_size=12, marker_line_width=1.5)
fig.update_layout(legend_orientation='h')
fig.show()

In [34]:
print(clf.predict(X_teste_iris))

print(np.array(Y_teste_iris))

[1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 2 0 0 0 2 2 2 0 1 0 1 1 1 2 1 1 2 2 2 0 2
 2 2 2 0 0 1 0 1]
[1 2 0 1 0 1 1 1 0 1 1 2 1 0 0 2 1 0 0 0 2 2 2 0 1 0 1 1 1 2 1 1 2 2 2 0 2
 2 2 2 0 0 1 0 1]
